<a href="https://colab.research.google.com/github/PlatiB/Deep_product_category_classification/blob/master/product_category_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 상품 카테고리 분류 딥러닝

In [0]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

import re


from keras.utils import np_utils
from keras.layers import Dense, BatchNormalization, Activation, Dropout
from keras import layers, models

import random as rd

from sklearn.metrics import accuracy_score

Using TensorFlow backend.


drive에서 데이터 로딩

In [0]:
from google.colab import drive

In [0]:
drive.mount('./now') #마운트 두번 잘못하면 꼬임.

In [0]:
#drive.unmount()

In [0]:
cd /content/now/My Drive/Colab Notebooks/product category classification

In [0]:
ls -al

In [0]:
data = pd.read_csv (r'product_raw_info.csv')
df = pd.DataFrame(data, columns= ['prod_name','prod_category','prod_price'])

In [0]:
print(df['prod_name'][:10])

# KoNLPy 사용하지 않고 자연어 직접 처리시 사용하는 블럭
## 물품정보 텍스트 분리 function

def tsplit(string, delimiters):
    """Behaves str.split but supports multiple delimiters."""
    
    delimiters = tuple(delimiters)
    stack = [string,]
    
    for delimiter in delimiters:
        for i, substring in enumerate(stack):
            substack = substring.split(delimiter)
            stack.pop(i)
            for j, _substring in enumerate(substack):
                stack.insert(i+j, _substring)
            
    return stack

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def seperate_string_number(string):
    previous_character = string[0]
    groups = []
    newword = string[0]
    for x, i in enumerate(string[1:]):
        if i.isalpha() and previous_character.isalpha():
            newword += i
        elif i.isnumeric() and previous_character.isnumeric():
            newword += i
        else:
            groups.append(newword)
            newword = i

        previous_character = i

        if x == len(string) - 2:
            groups.append(newword)
            newword = ''
    return groups



for name in df['prod_name'][:10]:
    productDescriptions = tsplit(name, [')','('])
    afterProductDescriptions = []
    
    for i, elem in enumerate(productDescriptions):
        # if hasNumbers(elem):
        seperateGroup = seperate_string_number(elem)

        for descriptor in seperateGroup:
            afterProductDescriptions.append(descriptor)
        # print(seperateGroup)

    print('productDescriptions : ', afterProductDescriptions)


In [0]:
# 한글 자연어 처리 라이브러리 KoNLPy 를 이용해 물품명 분석

!pip install konlpy
from konlpy.tag import Kkma
from konlpy.utils import pprint

kkma = Kkma()
for name in df['prod_name'][:10]:
    print(kkma.sentences(name)[0].split())
    pprint(kkma.sentences(name))


In [0]:
# bag of words functions

def replace_special_character(sentence):
    # my_new_string = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', sentence)
    my_new_string = re.sub('([^\s\w]|_)+', ' ', sentence)
    # print(my_new_string)
    return my_new_string


def tokenize_sentences(sentences):
    words = []
    for i, sentence in enumerate(sentences):
        # print(sentence)
        if i % 1000 == 0:
            print('tokenize... ', i, ' / ', sentences.size)
        w = extract_words(replace_special_character(sentence))
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

def extract_words(sentence):
    # ignore_words = ['a']
    # words = re.sub("[^w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
    # words_cleaned = [w.lower() for w in words if w not in ignore_words]

    kkma = Kkma()
    # kkma.sentences(sentence)[0].split()

    # return words_cleaned   
    return kkma.sentences(sentence)[0].split() 
    
def bagofwords(sentence, words):
    sentence_words = extract_words(sentence)
    # frequency word count
    bag = np.zeros(len(words))
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag)


In [0]:
# Words Bag 생성
#sentences = df['prod_name']
#vocabulary = tokenize_sentences(sentences)

#매번 생성시 시간이 많이 소요되어, 생성후 csv로 저장해둠
#vocaDataFrame = pd.DataFrame(vocabulary, columns=["word"])
#vocaDataFrame.to_csv('vocabulary.csv', index=False)

#vocabulary 로딩
vocaData = pd.read_csv (r'vocabulary.csv')
# print(vocaData)
vocaDataFrame = pd.DataFrame(vocaData, columns= ['word'])
type(vocaDataFrame)
vocabulary = vocaDataFrame['word'].tolist()
len(vocabulary)

5274

In [0]:
category = ['파이',
            '탄산음료',
            '쿠키',
            '케잌',
            '커피',
            '초콜릿',
            '차',
            '즉석식품',
            '주스',
            '주류',
            '젤리',
            '장난감',
            '의류',
            '음료수',
            '유제품',
            '위생용품',
            '원물간식',
            '에너지드링크',
            '애견용품',
            '아이스크림',
            '시리얼',
            '소시지',
            '생활용품',
            '생수',
            '사탕',
            '빵',
            '미용용품',
            '라면',
            '두유',
            '껌',
            '과채주스',
            '과자',
            '건강음료'
]

In [0]:
# 레이블데이터 숫자화
train_y = df['prod_category'].apply(category.index)
train_y = np.array(train_y)

# One-Hot 벡터 인코딩
train_y = np_utils.to_categorical(train_y)

In [0]:
print(train_y)
print(type(np.array(train_y)))
print(train_y.shape)

In [0]:
# train 데이터, BOW 수행하여 피쳐 추출 및 데이터 정제
listTrain_X = []
for i, prodName in enumerate(df['prod_name']):
    if i % 1000 == 0:
        print('bag of words processing... ', i, '/', len(df['prod_name']))
    listTrain_X.append(bagofwords(prodName, vocabulary))

train_x = np.asarray(listTrain_X)

In [0]:
#train_x = bagofwords(df['prod_name'][0], vocabulary)
#bagofwords(df['prod_name'][0], vocabulary).shape
print(train_x)
print(type(train_x))
print(train_x.shape)

In [0]:
print(train_x.shape, train_y.shape)

(38178, 5274) (38178, 33)


In [0]:
#트레이닝, 테스트, Validation 데이터 분리 80%, 10%, 10%
final_train_x, final_valid_x, final_test_x = np.split(train_x, [int(.8 * len(train_x)), int(.9 * len(train_x))])
final_train_y, final_valid_y, final_test_y = np.split(train_y, [int(.8 * len(train_y)), int(.9 * len(train_y))])

In [0]:
print(final_train_x.shape, final_train_y.shape, final_test_x.shape,  final_test_y.shape, final_valid_x.shape, final_valid_y.shape)

(30542, 5274) (30542, 33) (3818, 5274) (3818, 33) (3818, 5274) (3818, 33)


In [0]:
#결과 확인을 위해, 피쳐 추출, 레이블 수치화 하기전 데이터 저장
org_train_x, org_valid_x, org_test_x = np.split(np.array(df['prod_name']), [int(.8 * len(df['prod_name'])), int(.9 * len(df['prod_name']))])
org_train_y, org_valid_y, org_test_y = np.split(np.array(df['prod_category']), [int(.8 * len(df['prod_category'])), int(.9 * len(df['prod_category']))])

In [0]:
print(org_train_x.shape, org_valid_x.shape, org_test_x.shape,  org_train_y.shape, org_valid_y.shape, org_test_y.shape)

(30542,) (3818,) (3818,) (30542,) (3818,) (3818,)


In [0]:
#모델 클리어
keras.backend.clear_session()
model = models.Sequential()

In [0]:
model.add( Dense(4096, input_shape=(5274,), name = 'Hidden1', # activation = 'relu',
                kernel_initializer = 'glorot_uniform',
                bias_initializer = 'glorot_uniform'))

In [0]:
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.1))

In [0]:
model.add( Dense(4096, name = 'Hidden2', # activation = 'relu',
                kernel_initializer = 'glorot_uniform',
                bias_initializer = 'glorot_uniform'))

In [0]:
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.1))

In [0]:
model.add( Dense(512, name = 'Hidden3', # activation = 'relu',
                kernel_initializer = 'glorot_uniform',
                bias_initializer = 'glorot_uniform'))

In [0]:
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.1))

In [0]:
#output Layer
model.add(layers.Dense(33, name = 'OutputLayer',
                       activation = 'softmax',
                      kernel_initializer = 'glorot_uniform',
                      bias_initializer = 'glorot_uniform'))

In [0]:
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [0]:
from keras import optimizers

# 러닝 레잇 조절
adam = optimizers.Adam(lr=0.0001)

# 컴파일
model.compile(loss = 'categorical_crossentropy', optimizer = adam,
              metrics =['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss',
                  min_delta = 0, # 개선되고 있다고 판단하기 위한 최소 변화량
                  patience = 10, # 개선 없는 epoch 얼마나 기달려 줄거야?
                  verbose = 1
                  )

history = model.fit(final_train_x, final_train_y, epochs=10, batch_size = 2000,
                   validation_split = 0.2,
                   verbose = 1,
                   callbacks = [es])

In [0]:
model.summary()

In [0]:
#트레이닝 이력 확인

if not isinstance(history, dict):
    history = history.history

plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Accuracy : Training vs Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
ls -al

In [0]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
from keras import optimizers

# 러닝 레잇 조절
adam = optimizers.Adam(lr=0.0001)

model.compile(loss = 'categorical_crossentropy', optimizer = adam,
              metrics =['accuracy'])

In [0]:
performance_test = model.evaluate(final_valid_x, final_valid_y, batch_size = 100)

print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

3818/3818 [==============================] - 6s 2ms/step
Test Loss : 0.608403,  Test Accuracy : 93.740%


In [0]:
print(final_valid_x)

In [0]:
pred_test = model.predict(final_valid_x)
single_pred_test = pred_test.argmax(axis=1)
# final_test_x[2].shape

In [0]:
## 예측치를 확인해보기
id = rd.randrange(0,1000)
print('id = {}'.format(id))
#print('품명 = {}'.format(final_valid_x[id]))
print('품명 = {}'.format(org_valid_x[id]))
print('이 상품은 : {} 입니다.'.format(category[final_valid_y.argmax(axis=1)[id]]))
print('모델의 예측 : {}'.format(category[single_pred_test[id]]))


id = 973
품명 = 크라운)식이섬유바32g
이 상품은 : 초콜릿 입니다.
모델의 예측 : 초콜릿


In [0]:
#틀린것만 보기
err_count = 0
for i, predict in enumerate(single_pred_test):
    if category[predict] != category[final_valid_y.argmax(axis=1)[i]]:
        err_count += 1
        print('id = {}'.format(i))
        print('품명 = {}'.format(org_valid_x[i]))
        print('이 상품은 : {} 입니다.'.format(category[final_valid_y.argmax(axis=1)[i]]))
        print('모델의 예측 : {}'.format(category[single_pred_test[i]]))

print('오류건수 : {}'.format(err_count))

id = 71
품명 = 비타500캔
이 상품은 : 음료수 입니다.
모델의 예측 : 건강음료
id = 81
품명 = 몽베스트생수500
이 상품은 : 생수 입니다.
모델의 예측 : 파이
id = 82
품명 = 몽베스트생수1L
이 상품은 : 생수 입니다.
모델의 예측 : 생활용품
id = 113
품명 = 천호석류70ml
이 상품은 : 주스 입니다.
모델의 예측 : 음료수
id = 119
품명 = 썬키스트허니유자280병
이 상품은 : 음료수 입니다.
모델의 예측 : 차
id = 124
품명 = 천호도라지배즙70ml
이 상품은 : 건강음료 입니다.
모델의 예측 : 아이스크림
id = 126
품명 = 영진구론산오리지날150ml
이 상품은 : 음료수 입니다.
모델의 예측 : 건강음료
id = 150
품명 = 천호웰스H도라지110ml
이 상품은 : 건강음료 입니다.
모델의 예측 : 주스
id = 156
품명 = 매일아몬드브리즈바나나190ml
이 상품은 : 두유 입니다.
모델의 예측 : 음료수
id = 184
품명 = 마다가스카르바닐라콘
이 상품은 : 아이스크림 입니다.
모델의 예측 : 과자
id = 192
품명 = 웰그린허니제주한라봉
이 상품은 : 커피 입니다.
모델의 예측 : 주스
id = 193
품명 = 와사비맛김3입
이 상품은 : 원물간식 입니다.
모델의 예측 : 즉석식품
id = 245
품명 = 치킨팝닭강정맛
이 상품은 : 과자 입니다.
모델의 예측 : 즉석식품
id = 248
품명 = 밀크초코바나나킥
이 상품은 : 과자 입니다.
모델의 예측 : 유제품
id = 250
품명 = 빅빅초코바
이 상품은 : 초콜릿 입니다.
모델의 예측 : 아이스크림
id = 252
품명 = 록키마운틴마시멜로
이 상품은 : 사탕 입니다.
모델의 예측 : 젤리
id = 260
품명 = 스니커즈헤이즐넛싱글
이 상품은 : 초콜릿 입니다.
모델의 예측 : 의류
id = 261
품명 = 도브헤이즐넛1500
이 상품은 : 아이스크림 입니다.
모델의 예측 : 초콜릿
id = 281
품명 = 오키오망고구